In [26]:
import pandas as pd
import numpy as np 

In [27]:
database_df = pd.read_csv('C:/Users/lgrah/Desktop/VSC Projects/DoorDash Case Study/Database Report - Database pull.csv')

In [28]:
# Identify every column that is not delivery ID and convert into dollars
columns_to_convert = database_df.columns.difference(['Delivery ID'])
database_df[columns_to_convert] = database_df[columns_to_convert] / 100

In [29]:
# Remove "(in cents)" for easier reading
database_df.columns = [col.replace("(in cents)", "").strip() for col in database_df.columns]

In [30]:
# Convert discount,refund, stripe cost columns to negative values for user friendly analysis
negative_cols = ['Discount','Refund','Stripe Cost']
database_df[negative_cols] = database_df[negative_cols] *-1

In [31]:
total_order_amt_list = ['Food Subtotal', 'Tax','Delivery Fee','Tips','Discount']
database_df['Total Order Amount'] = database_df[total_order_amt_list].sum(axis = 1)

In [32]:
database_df.head()

,Delivery ID,Food Subtotal,Tax,Delivery Fee,Tips,Commission,Inflation,Discount,Refund,Stripe Cost,Total Order Amount
0,1,232.10,23.44,12.0,26.79,16.25,35.80,-0.0,-0.0,-7.89,294.33
1,2,179.13,17.22,12.0,19.68,26.87,17.72,-6.0,-0.0,-5.81,222.03
2,3,245.00,24.33,12.0,41.70,17.15,33.00,-0.0,-0.0,-8.48,323.03
3,4,27.47,2.53,6.0,1.44,4.12,1.48,-0.0,-0.0,-1.19,37.44
4,5,35.99,3.24,6.0,1.79,6.48,0.00,-0.0,-0.0,-1.38,47.02


In [33]:
database_df['Accrued Dasher Delivery Payable'] = database_df['Delivery Fee'].apply(lambda x: x if x <= 7 else 7)
database_df['Delivery Fee Revenue'] = database_df['Delivery Fee'].apply(lambda x: x -7 if x > 7 else 0)
database_df['Commission Revenue'] = database_df['Commission']

In [34]:
database_df.head()

,Delivery ID,Food Subtotal,Tax,Delivery Fee,Tips,Commission,Inflation,Discount,Refund,Stripe Cost,Total Order Amount,Accrued Dasher Delivery Payable,Delivery Fee Revenue,Commission Revenue
0,1,232.10,23.44,12.0,26.79,16.25,35.80,-0.0,-0.0,-7.89,294.33,7.0,5.0,16.25
1,2,179.13,17.22,12.0,19.68,26.87,17.72,-6.0,-0.0,-5.81,222.03,7.0,5.0,26.87
2,3,245.00,24.33,12.0,41.70,17.15,33.00,-0.0,-0.0,-8.48,323.03,7.0,5.0,17.15
3,4,27.47,2.53,6.0,1.44,4.12,1.48,-0.0,-0.0,-1.19,37.44,6.0,0.0,4.12
4,5,35.99,3.24,6.0,1.79,6.48,0.00,-0.0,-0.0,-1.38,47.02,6.0,0.0,6.48


# Calculation of funds in Stripe Merchant Account (Accounts Receivable)

In [35]:
# Starting Value of Stripe Merchant account (Accounts Rec) at 0
stripe_merchant_account_ar = 0

""" Accounting for transactions at point of sale for each Delivery ID """

# Accounting for Restaurant Sales
interim_merchant_sales = database_df['Food Subtotal'].sum() #Credit
stripe_merchant_account_ar = stripe_merchant_account_ar + interim_merchant_sales

# Accounting for Sales Tax
accrued_sales_tax = database_df['Tax'].sum() # Credit
stripe_merchant_account_ar = stripe_merchant_account_ar + accrued_sales_tax

# Accounting for Delivery Fees
interim_delivery_fees = database_df['Delivery Fee'].sum() # Credit
stripe_merchant_account_ar = stripe_merchant_account_ar + interim_delivery_fees

# Accounting for Dasher Tips
accrued_dasher_tips = database_df['Tips'].sum() # Credit
stripe_merchant_account_ar = stripe_merchant_account_ar + accrued_dasher_tips

# Accounting for Restaurnt Sales Discounts
sales_discounts = database_df['Discount'].sum() # Debit (Contra Asset)
stripe_merchant_account_ar = stripe_merchant_account_ar + sales_discounts

print("Total Cash in Stripe Merchant Account: $", stripe_merchant_account_ar)

Total Cash in Stripe Merchant Account: $ 1085933.61


# Acrruals for Restaurant Transactions

In [36]:
accounts_receivable = 0

# Payment of Delivery Fees for Dashers
dasher_delivery_fees_payable = database_df['Accrued Dasher Delivery Payable'].sum() # Credit
interim_delivery_fees = interim_delivery_fees - dasher_delivery_fees_payable # Reduce Interim Fees account by the amount owed to Dashers
stripe_merchant_account_ar = stripe_merchant_account_ar - dasher_delivery_fees_payable # Dashers are paid out using DoorDash's Stripe Merchant Account

# Recognition of Excess Delivery Fee Revenue
delivery_fee_revenue = database_df['Delivery Fee Revenue'].sum() # Credit
interim_delivery_fees = interim_delivery_fees - delivery_fee_revenue # Reduce Interim Fees account by the amount of DoorDash revenue through execess delivery fees
accounts_receivable = accounts_receivable + delivery_fee_revenue # Debit AR due to Stripe Accounts funds not transferring to Bank Account yet

# Accounting for Commission Fees from Partner Merchants
commission_fee_revenue = database_df['Commission Revenue'].sum() # Credit Commission Revenue
accounts_receivable = commission_fee_revenue # Debit Accounts Receivable 

# Accounting for Refunds on transactions
sales_returns_allowances = database_df['Refund'].sum() # Debit (Contra Revenue) - Assumed this amount will be credited back to the customer
accounts_receivable = accounts_receivable - sales_returns_allowances # Credit - Reducing AR for sales return

# Accounting for Payout of Tips
stripe_merchant_account_ar = stripe_merchant_account_ar - accrued_dasher_tips # Reduce Stripe Merchant Account by Total Tips amount
accrued_dasher_tips = accrued_dasher_tips - accrued_dasher_tips # Reverse our accrued tips because they were paid out to dashers

# Accounting for Stripe Processing Fees - Assumption that we accrue the fees and have not paid out
merchant_fees_expense = database_df['Stripe Cost'].sum() # Debit
merchant_fees_payable = merchant_fees_expense # Credit

In [38]:
print("Total Cash in Stripe Merchant Account at End of Month X: $", stripe_merchant_account_ar)
print("Ending Accounts Receivable: $", accounts_receivable)
print("Ending Interim Delivery Fees Account: $", interim_delivery_fees)
print("Ending Dasher Delivery Fees Payable Account: $", dasher_delivery_fees_payable)
print("Ending Delivery Fees Revenue Account: $", delivery_fee_revenue)
print("Ending Commission Fees Revenue Account: $", commission_fee_revenue)
print("Ending Sales Returns & Allowances Account: $", sales_returns_allowances)
print("Ending Merchant Fees Expense Account: $", merchant_fees_expense)
print("Ending Merchant Fees Payable Account: $", merchant_fees_payable)


Total Cash in Stripe Merchant Account at End of Month X: $ 867694.05
Ending Accounts Receivable: $ 79292.37
Ending Interim Delivery Fees Account: $ 1.546140993013978e-11
Ending Dasher Delivery Fees Payable Account: $ 129364.0
Ending Delivery Fees Revenue Account: $ 6671.580000000001
Ending Commission Fees Revenue Account: $ 67469.26
Ending Sales Returns & Allowances Account: $ -11823.109999999999
Ending Merchant Fees Expense Account: $ -32353.82
Ending Merchant Fees Payable Account: $ -32353.82


In [39]:
database_df.to_csv('door_dash_database_month_x.csv')